### Instalando las librerias necesarias

Para la correcta ejecucion del proyecto primero se deberan de instalar las librerias usadas para la implementacion del mismo.

In [2]:
!pip install anytree pandas
!pip install pytholog
!apt install swi-prolog
!pip install pykeen
!pip install -U scikit-fuzzy


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
"apt" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


### Implementacion del agente

Se importan las librerías relevantes.

In [3]:
import numpy as np
import pandas as pd
from anytree import NodeMixin, RenderTree
import pytholog as pl 
import math
import itertools
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

Este metodo convierte cualquier lista de tareas sin orden en un arbol, para que el algoritmo de busqueda pueda luego seleccionar el mejor camino dentro del arbol.
Como entrada a modo de ejemplo pondremos un CSV de tareas sin orden, y a la salida el agente habrá ordenado las tareas en un arbol.

In [4]:
#Ordenar tareas en modo de arbol.

df = pd.read_csv("tasksUnordered.csv") # leer el archivo csv original

df = df.sort_values(by=["TimeStart"]) # ordenar las tareas por su tiempo de inicio

parents = {} #diccionario para almacenar los padres de cada tarea

# recorrer cada tarea
for i, row in df.iterrows():
    # encontrar el tiempo de finalización de la tarea actual
    current_end_time = row["TimeStart"] + row["AvgTime"]
    # inicializar la distancia más cercana como infinito
    closest_distance = float("inf")
    # inicializar el padre más cercano como None
    closest_parent = None
    # buscar el padre más cercano en las tareas anteriores
    for j in range(i):
        # encontrar el tiempo de finalización de la tarea anterior
        previous_end_time = df.iloc[j]["TimeStart"] + df.iloc[j]["AvgTime"]
        # si la tarea anterior termina antes de que comience la tarea actual
        # y la distancia entre la tarea anterior y la actual es menor que la distancia más cercana actual
        # actualizar el padre más cercano y la distancia más cercana
        if previous_end_time <= row["TimeStart"] and (row["TimeStart"] - previous_end_time) < closest_distance:
            closest_distance = row["TimeStart"] - previous_end_time
            closest_parent = int(df.iloc[j]["TaskID"])
    # si se encontró un padre cercano, agregarlo al diccionario de padres
    if closest_parent is not None:
        parents[int(row["TaskID"])] = int(closest_parent)

# agregar la columna Parent al dataframe
df["Parent"] = df["TaskID"].map(parents)

# guardar el dataframe en un nuevo archivo csv
df.to_csv("tasksOrdered.csv", index=False)


Este fragmento de código calcula la urgencia de cada tarea usando Redes de Creencias. anteriormente la urgencia se calculaba holisticamente, pero mediante este metodo logramos que tenga valores mas logicos y coherentes para ser usados como insumo luego.

In [5]:
"""
Código que calcula la urgencia de cada tarea usando Redes de Creencias.

Definir las funciones de creencia para cada factor:
"""

def creencia_avg_time(avg_time:int) -> float:
    """
    Creencia triangular para que las tareas con tiempo promedio de terminacion
    mas bajos sean prioriozadas. 
    
    Args:
        avg_time (int): Tiempo promedio que se demora en ejecutar y completar la tarea (minutos).
        
    Returns:
        float: Peso de la tarea de acuerdo con su tiempo promedio de terminacion, 
        entre mas se demore mas pequeño va a ser el peso que esta posea.  
    """
    if avg_time < 10:
        return 0.2
    elif avg_time < 20:
        return (avg_time - 10) / 10
    elif avg_time < 30:
        return 1 - (avg_time - 20) / 10
    else:
        return 0.2



def creencia_deadline(deadline:int) -> float:
    """
    Funcion de Creencia inversa para que las tareas que tengan una fecha limite
    mas pequeña (mas cerca del limite) sean priorizadas.
    Los valores más bajos tendran más prioridad y los valores más altos tendran menos prioridad.
    
    Args:
        deadline (int): Numero de dias que faltan para la fecha limite de la tarea. 
    
    Returns:
        float: Peso de la tarea de acuerdo con su fecha limite, 
        entre mas pequeña sea la fecha limite mas grande va a ser el peso de la tarea. 
    """
    if deadline < 1:
        return 1
    elif deadline < 5:
        return 1 - (deadline - 1) / 4
    elif deadline < 10:
        return 0.5 - (deadline - 5) / 10
    else:
        return 0



def creencia_is_hobby(is_hobby:bool) -> float:
    """
    Funcion de creencia que baja un poco la prioridad de una tarea si esta es un hobby.
    
    Args:
        is_hobby (bool): Si la tarea es o no un hobby.
        
    Returns:
        float: Peso de la tarea si la misma es o no un hobby.
    """
    if is_hobby:
        return 0.8
    else:
        return 1



def calcular_urgencia(avg_time:int, deadline:int, is_hobby:bool) -> float:
    """
    Construccion de una red de creencias que combine las funciones de creencia:
    creencia_avg_time, creencia_deadline y creencia_is_hobby usando una regla del producto.
    
    Args:
        avg_time (int): Tiempo promedio que se demora en ejecutar la tarea
        deadline (int): Numero de dias que faltan para la fecha limite de la tarea. 
        is_hobby (bool): Si la tarea es o no un hobby.
        
    Returns:
        float: Peso total asignado a la tarea, mediante el calculo de los diferentes
        pesos de sus artibutos avg_time, deadline e isHobby por medio de las funciones
        creencia_avg_time, creencia_deadline y creencia_is_hobby usando una regla del producto.
    """
    creencia_avg_time_res = creencia_avg_time(avg_time)
    creencia_deadline_res = creencia_deadline(deadline)
    creencia_is_hobby_res = creencia_is_hobby(is_hobby)
    urgencia_calc = creencia_avg_time_res * creencia_deadline_res * creencia_is_hobby_res
    return urgencia_calc



def escalar_urgencia(urgencia_calc:float) -> float:
    """
    Escalar la urgencia dentro de un intervalo de valores aceptados; [1,10].

    Args:
        urgencia_calc (float): Peso total asignado a la tarea, mediante el calculo de los diferentes
                               pesos de sus artibutos avg_time, deadline e isHobby por medio de las funciones
                               creencia_avg_time, creencia_deadline y creencia_is_hobby usando una regla del producto.

    Returns:
        float: Urgencia escalada al intervalo definido: maximo la urgencia puede tomar un valor de 10
        y minimo la urgencia puede tomar una valor de 1, [1,10].
    """
    urgencia_min = 1
    urgencia_max = 10
    urgencia = urgencia_min + ((urgencia_calc - 0) / (1 - 0)) * (urgencia_max - urgencia_min)
    return round(urgencia, 2)



df = pd.read_csv("tareas.csv") # Leer el CSV

# Calcular la urgencia para cada tarea y escalar el valor entre 1 y 10 con máximo 2 espacios decimales
df["Urgency"] = df.apply(lambda row: escalar_urgencia(calcular_urgencia(row["AvgTime"], row["Deadline"], row["IsHobby"])), axis=1)

df.to_csv("tareas_con_urgencia.csv", index=False) # Guardar el CSV con la nueva columna

Aquí vamos a crear la clase Task y TaskClass, que son los ladrillos constructores, las unidades mínimas con las que estaremos analizando cada tarea y armando nuestras soluciones.

In [6]:
class Task:
    def __init__(self, id:int, taskType:int, name:str, avgTime:int, 
                 timeStart:int, deadline:int, urgency:float, location:str, 
                 isHobby:bool, recurrency:int, bestOn:str, involves:str):
        """
        Constructor del objeto tarea.

        Args:
            id (int): Identificador unico de la tarea
            taskType (int): Categoria de la tarea a completar.
                1- Alimentación​,
                2- Aseo​,
                3- Actividad de Mascota​,
                4- Estudio​,
                5- Salud Física​,
                6- Plantas​,
                7- Gustos​,
                8- Actividad Social​,
                9- Actividad de Autocuidado y Orden.
            name (str): Nombre de la tarea
            avgTime (int): Tiempo promedio que se demora en ejecutar y completar la tarea (minutos).
            timeStart (int): _description_
            deadline (int): Numero de dias que faltan para la fecha limite de la tarea. 
            urgency (float): Nivel de prioridad
            location (str): Lugar fisico en el cual se realizara la tarea.
            isHobby (bool): Si la tarea es o no un hobby.
            recurrency (int): Cuantos dias en la semana suele repetirse la tarea.
            bestOn (str): Mejor en la mañana o en la tarde.
            involves (str): Sujetos terceros que involucra la tarea.
        """
        self.id=id
        self.taskType=taskType
        self.name=name
        self.avgTime=avgTime
        self.timeStart=timeStart
        self.deadline=deadline
        self.location=location
        self.urgency=urgency
        self.isHobby=isHobby
        self.recurrency=recurrency
        self.bestOn=bestOn
        self.involves=involves
        self.prioridad=""
        self.onPath=False
    
    
    
    def __str__(self) -> str:
        """
        To String del objeto Task. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Task): Instancia de la clase Task.
        
        Returns:
            str: Representacion como texto del objeto Task.
        """
        return f"id: {self.id}, taskType: {self.taskType}, name: {self.name}, avgTime: {self.avgTime}, timeStart:{self.timeStart},deadline:{self.deadline},location:{self.location} urgency: {self.urgency}, isHobby: {self.isHobby}, recurrency:{self.recurrency}, bestOn:{self.bestOn} "



class TaskClass(Task, NodeMixin):  # Add Node feature
    def __init__(self,id:int, taskType:int, name:str, avgTime:int, 
                 timeStart:int, deadline:int, urgency:float, location:str, 
                 isHobby:bool, recurrency:int, bestOn:str, involves:str, 
                 parent:NodeMixin=None, children:NodeMixin=None):
        """
        Constructor del objeto Tarea como nodo de un arbol.

        Args:
            id (int): Identificador unico de la tarea
            taskType (int): Categoria de la tarea a completar.
                1- Alimentación​,
                2- Aseo​,
                3- Actividad de Mascota​,
                4- Estudio​,
                5- Salud Física​,
                6- Plantas​,
                7- Gustos​,
                8- Actividad Social​,
                9- Actividad de Autocuidado y Orden.
            name (str): Nombre de la tarea
            avgTime (int): Tiempo promedio que se demora en ejecutar y completar la tarea (minutos).
            timeStart (int): _description_
            deadline (int): Numero de dias que faltan para la fecha limite de la tarea. 
            urgency (float): Nivel de prioridad
            location (str): Lugar fisico en el cual se realizara la tarea.
            isHobby (bool):  Si la tarea es o no un hobby.
            recurrency (int): Cuantos dias en la semana suele repetirse la tarea.
            bestOn (str): Mejor en la mañana o en la tarde.
            involves (str): Sujetos terceros que involucra la tarea.
            parent (NodeMixin, optional): _description_. Defaults to None.
            children (NodeMixin, optional): _description_. Defaults to None.
        """
        super().__init__(id, taskType, name, avgTime, timeStart, deadline, urgency, location, isHobby, recurrency, bestOn, involves) #llamada al constructor padre
        self.parent = parent
        self.onPath=False
        if children:
            self.children = children

Esta es la clase auxiliar choice que nos sirve en el recorrido de la mejor solución.

In [7]:
class Choice():
    def __init__(self, move, urgency):
        self.move = move
        self.urgency = urgency



    def __str__(self):
        """
        To String del objeto Choice. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Choice): Instancia de la clase Choice.
        
        Returns:
            str: Representacion como texto del objeto Choice.
        """
        return self.move + ": " + str(self.urgency)
    

Aquí tenemos el constructor de nuestro agente y clases relacionadas con su estructura básica.

In [23]:
class ScheduleAgent:
    def __init__(self, name:str, age:int, gender:str, taskList:pd.DataFrame, comments):
        """
        Constructor del objeto agente

        Args:
            name (str): Nombre del usuario
            age (int): Edad del usuario
            gender (str): Genero del usuario
            taskList (pd.DataFrame): Lista de tareas del usuario
            comments (_type_): _description_
        """
        self.name = name
        self.age = age
        self.gender = gender
        self.taskList = taskList
        self.notification = comments
        self.comments = comments
        self.rootNode = self.buildTree() #creo el arbol al inicializar el objeto
        self.miniMaxSchedule = self.getMiniMaxSchedule() #creo el mejor horario segun recorrido minimMax al inicializar el objeto
    
    
  
    def getNodesList(self) -> list:
        """
        Crear una lista de nodos del arbol de tareas de acuerdo a las
        tareas almacenadas en un archivo CSV.

        Args:
            self (ScheduleAgent): Instancia de la clase ScheduleAgent.

        Returns:
            list: Lista de nodos del arbol de tareas, cada nodo representa una tarea.
        """
        nodeList = []
        for i in range(0, len(self.taskList["Name"])):
            nodeList.append(TaskClass(
                id=self.taskList["TaskID"][i],
                taskType = self.taskList["TaskType"][i],
                name = self.taskList["Name"][i],
                avgTime = self.taskList["AvgTime"][i],
                timeStart = self.taskList["TimeStart"][i],
                deadline = self.taskList["Deadline"][i],
                urgency = self.taskList["Urgency"][i],
                location = self.taskList["Location"][i],
                isHobby = self.taskList["IsHobby"][i],
                recurrency = self.taskList["Recurrency"][i],
                bestOn = self.taskList["BestOn"][i],
                involves = self.taskList["Involves"][i],
            ))
        return nodeList
  
  
  
    def setParentNodes(self, nodeList:list) -> TaskClass:
        """
        Colocar a cada uno de los nodos del arbol de tareas su respectivo nodo padre.

        Args:
            self (ScheduleAgent): Instancia de la clase ScheduleAgent.
            nodeList (list): Lista de nodos del arbol de tareas, cada nodo representa una tarea.

        Returns:
            TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
        """
        for i in range(1, len(nodeList)):
            node = nodeList[i]
            node.parent = nodeList[(int(self.taskList["Parent"][i]))-1]
        return nodeList[0] #retorno root del arbol ya que desde root lo puedo recorrer todo

  
  
    def buildTree(self) -> TaskClass: 
        """
        Armar el arbol de tareas.
        En primera instancia crea una lista de nodos de tareas para 
        despues asignarles su respectivo nodo padre.

        Args:
            self (ScheduleAgent): Instancia de la clase ScheduleAgent.

        Returns:
            TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
        """
        nodeList = self.getNodesList()
        rootNode = self.setParentNodes(nodeList)
        return rootNode #retorno root del arbol ya que desde root lo puedo recorrer todo
        
    
    
    def printTaskTree(self) -> None:
        """
        Imprimir por completo el arbol de tareas. 
        
        Args:
            self (ScheduleAgent): Instancia de la clase ScheduleAgent.
        """
        for pre, fill, node in RenderTree(self.rootNode):
            print("%s%s" % (pre, node.name))
  
  
  
    def miniMaxChoice(self, rootNode:Task, isMax:bool) -> Choice:
        """
        Recorrer un nodo del arbol, hallar los valores de sus hijos y
        decidir por cual rama seguir (izquierda, derecha).

        Args:
            self (ScheduleAgent): Instancia de la clase ScheduleAgent.
            rootNode (Task): Nodo padre del cual se va a recorrer sus hijos
            isMax (bool): ?

        Returns:
            Choice: Eleccion de rama y nodo del algortimo miniMax.
        """
        children = rootNode.children
        try:
            l_choice = self.miniMaxChoice(children[0], not isMax)
            r_choice = self.miniMaxChoice(children[1], not isMax)

            if (isMax):
                if (l_choice.urgency > r_choice.urgency):
                    return Choice("left", l_choice.urgency)
                else:
                    return Choice("right", r_choice.urgency)
            else:
                if (l_choice.urgency < r_choice.urgency):
                    return Choice("left", l_choice.urgency)
                else:
                    return Choice("right", r_choice.urgency) 
        except IndexError:
            return Choice("end", self.rootNode.urgency)
    
    
    
    def runMiniMaxSearch(self) -> dict:
        """
        Correr el algoritmo de busqueda miniMax para determinar la mayor cantidad de tareas de la mayor urgencia posible
        a completar en el minimo tiempo posible.

        Returns:
            dict: Horario hallado al recorrer el arbol por medio del algoritmo miniMax.
        """
        bestSchedule = {}
        def add_task(node): #la creo en el scope local ya que no se va a usar en otras cosas del agente
            bestSchedule[node.name] = node
        isMax = True
        currentNode = self.rootNode
        add_task(currentNode) #agrego al cronograme el nodo padre del arbol
        currentNode.onPath = True
        while(True):
            chosenTask = self.miniMaxChoice(currentNode, isMax)
            if (chosenTask.move == 'left'):
                currentNode.children[0].onPath = True
                currentNode = currentNode.children[0]
                add_task(currentNode)
            elif (chosenTask.move == 'right'):
                currentNode.children[1].onPath = True
                currentNode = currentNode.children[1]
                add_task(currentNode) 
            elif (chosenTask.move == 'end'):
                break
        return bestSchedule
                


    def getMiniMaxSchedule(self) -> pd.DataFrame:
        """
        Convertir el horario hallado al recorrer el arbol en un DataFrame.

        Returns:
            pd.DataFrame: Horario hallado al recorrer el arbol por medio del algoritmo miniMax.
        """
        #la creo en el scope local ya que no se va a usar en otras cosas del agente
        def miniMaxScheduleToDf(bestSchedule:dict):
            schedule_df = pd.DataFrame(columns = ["Avg Time to Complete",
                                                "Task Urgency",
                                                "Is it Hobby",
                                                "Task Location",
                                                "Task Recurrency"],
                                    index = list(bestSchedule.keys()))
            for key in bestSchedule.keys():
                task_dict = bestSchedule[key].__dict__
                schedule_df.loc[key, 'Avg Time to Complete'] = task_dict['avgTime']
                schedule_df.loc[key,'Task Urgency'] = task_dict['urgency']
                schedule_df.loc[key,'Is it Hobby'] = key+' is a Hobby' if task_dict['isHobby'] else key+' is not a Hobby'
                schedule_df.loc[key,'Task Location'] = 'Uknown Location' if task_dict['location'] is None else task_dict['location']
                schedule_df.loc[key,'Task Recurrency'] = task_dict['recurrency']
        
            return schedule_df
        bestScheduleDict = self.runMiniMaxSearch()
        return miniMaxScheduleToDf(bestScheduleDict)        
    

Inicializamos el agente con un usuario ejemplo y calculamos la mejor solución para su horario.

In [20]:
#inicializando el agente para el usuario Ricardo
myAgent = ScheduleAgent(name="Ricardo",
                        age=36,
                        gender="Male",
                        taskList=pd.read_csv('agentTaskList.csv'),
                        comments = "")

In [21]:
myAgent.miniMaxSchedule

,Avg Time to Complete,Task Urgency,Is it Hobby,Task Location,Task Recurrency
desayunar,40,5,desayunar is not a Hobby,casa,7
sacar al perro,30,5,sacar al perro is not a Hobby,casa,7
regar plantas,15,7,regar plantas is not a Hobby,casa,2
jugar con el perro,30,6,jugar con el perro is not a Hobby,casa,6


Creamos estas estructuras de tipo diccionario para poder manejar las Tareas con mas facilidad.

In [ ]:
# PARA ORGANIZAR LAS TAREAS SEGUN SUS ATRIBUTOS.

dictTasks = {}
dictHobbies = {}
dictHabits = {}

Se añade la base de conocimiento al archivo, creada a partir del txt que se llena con la información de las tareas, y se llenan los diccionarios anteriores.

In [ ]:
# AÑADIR LAS TAREAS DE LA ESTRUCTURA ANYTREE AL ARCHIVO DE LOGICA DE PROLOG

# reinicia entre corridas el txt, aqui se ponen la reglas basicas de la logica del agente
with open("logica_agente.txt", "w") as archivo:
    archivo.write("sameLocation(X,Y) :- inLocation(X,Z1), inLocation(Y,Z2), Z1 == Z2 -> sameLocation(X,Y)." + '\n')
    archivo.write("differentLocation(X,Y) :- inLocation(X,Z1), inLocation(Y,Z2), Z1 \= Z2 -> differentLocation(X,Y)." + '\n')
    archivo.write("differentDuration(X,Y) :- taskDuration(X,D1), taskDuration(Y,D2), D1 \= D2 -> differentDuration(X,Y)." + '\n')
    archivo.write("higherUrgency(X,Y) :- hasUrgency(X,U1), hasUrgency(Y,U2), U1 > U2 -> higherUrgency(X,Y)." + '\n')
    archivo.write("isDailyTask(X) :- taskRecurrency(X,R), R >= 7 -> isDailyTask(X)." + '\n')
    archivo.write("isWeeklyTask(X) :- taskRecurrency(X,R), R < 7, R > 1 -> isWeeklyTask(X)." + '\n')
    archivo.write("isMonthlyTask(X) :- taskRecurrency(X,R), R == 1 -> isMonthlyTask(X)." )
archivo.close()

with open('logica_agente.txt', mode='a') as archivo:
    archivo.write("\nisAgent("+myAgent.name+")" +'\n')
archivo.close()


# METODO PARA AÑADIR LOS TASKS
def addAllTasks(node):
    with open('logica_agente.txt', mode='a') as archivo:

        dictTasks[node.name] = node
        archivo.write("isTask("+node.name + ")" + '\n')
        archivo.write("inLocation("+node.name + "," + node.location +")" + '\n')
        archivo.write("hasDeadline("+node.name + "," + str(node.deadline) +")" + '\n')
        archivo.write("hasUrgency("+node.name + "," + str(node.urgency) +")" + '\n')
        archivo.write("isBestOn("+node.name + "," + node.bestOn +")" + '\n')
        if node.onPath == True:
            archivo.write("inTodaysSchedule("+node.name + ")" + '\n')
        if node.involves != "" and node.involves != np.nan:
            archivo.write("involvesWho("+node.name + "," + str(node.involves) +")" + '\n')
        
        for node in node.descendants:
            if isinstance(node, Task):
                    dictTasks[node.name] = node
                    archivo.write("isTask("+node.name + ")" + '\n')
                    archivo.write("inLocation("+node.name + "," + node.location +")" + '\n')
                    archivo.write("hasDeadline("+node.name + "," + str(node.deadline) +")" + '\n')
                    archivo.write("hasUrgency("+node.name + "," + str(node.urgency) +")" + '\n')
                    archivo.write("isBestOn("+node.name + "," + node.bestOn +")" + '\n')
                    if node.onPath == True:
                        archivo.write("inTodaysSchedule("+node.name + ")" + '\n')
                    if node.involves != "" and node.involves != np.nan:
                        archivo.write("involvesWho("+node.name + "," + str(node.involves) +")" + '\n')
                    
    archivo.close()

# METODO PARA FILTRAR LAS ACTIVIDADES QUE SON HOBBIES
def addHobbies(node):
    with open('logica_agente.txt', mode='a') as archivo:
        for node in node.descendants:
            if isinstance(node, Task):
                if node.isHobby == True: 
                    dictHobbies[node.name] = node
                    archivo.write("isHobby("+node.name + ")" + '\n')
    archivo.close()


# METODO PARA FILTRAR LAS ACTIVIDADES QUE SON HABITOS
def addHabits(node):
    with open('logica_agente.txt', mode='a') as archivo:
        for node in node.descendants:
            if isinstance(node, Task):
                if node.recurrency >= 5: 
                    dictHabits[node.name] = node
                    archivo.write("isHabit("+node.name + ")" + '\n')
    archivo.close()
                
               

addAllTasks(myAgent.rootNode)
addHobbies(myAgent.rootNode)
addHabits(myAgent.rootNode)



# Base de conocimiento
KB2 = pl.KnowledgeBase("KB2")
KB2.from_file("logica_agente.txt")

facts and rules have been added to KB2.db


Este es un ejemplo de como quedaron llenos los diccionarios anteriormente descritos.

In [ ]:
print("Tasks: " + str(dictTasks))
print(str(len(dictTasks)))
print(dictTasks['bañarse'])
print(dictTasks['bañarse'].location)
print("")
print("Hobbies: " + str(dictHobbies))
print("")
print("Habits: " + str(dictHabits))

Tasks: {'desayunar': <__main__.TaskClass object at 0x12ce74460>, 'bañarse': <__main__.TaskClass object at 0x12ce74a60>, 'hacer la tarea': <__main__.TaskClass object at 0x12ce77a60>, 'almorzar': <__main__.TaskClass object at 0x12ce75000>, 'hacer ejercicio': <__main__.TaskClass object at 0x12ce74910>, 'estudiar para el parcial': <__main__.TaskClass object at 0x12ce74820>, 'escuchar musica': <__main__.TaskClass object at 0x12ce77190>, 'leer documentacion': <__main__.TaskClass object at 0x12ce75de0>, 'sacar al perro': <__main__.TaskClass object at 0x12ce75ab0>, 'hacer estiramientos': <__main__.TaskClass object at 0x12ce77430>, 'ver netflix': <__main__.TaskClass object at 0x12ce76200>, 'reunirse con amigos': <__main__.TaskClass object at 0x12ce77be0>, 'regar plantas': <__main__.TaskClass object at 0x12ce74eb0>, 'ordenar el cuarto': <__main__.TaskClass object at 0x12ce747c0>, 'jugar con el perro': <__main__.TaskClass object at 0x12ce77c70>}
15
id: 2, taskType: 2, name: bañarse, avgTime: 45, 

Estos son ejemplos de queries con nuestra base de conocimiento recientemente creada.

In [ ]:
# Ejemplos de Queries

# Queries sobre nuestros datos

print("Preguntamos sobre una tarea que no esta en el dataframe")
print(KB2.query(pl.Expr("isTask(desayunar)")))

print("Preguntamos datos sobre una tarea que sí esta en el dataframe")
print(KB2.query(pl.Expr("isTask(almorzar)")))

print("Preguntamos por clasificaciones")
print(KB2.query(pl.Expr("isHobby(almorzar)")))
print(KB2.query(pl.Expr("isHabit(almorzar)")))

print("Preguntamos datos sobre relaciones entre tareas")
print(KB2.query(pl.Expr("sameLocation(almorzar,regar plantas)")))
print(KB2.query(pl.Expr("sameLocation(desayunar,regar plantas)")))


print("Distinguimos que tareas son las que se agregaron al horario de hoy")
print(KB2.query(pl.Expr("inTodaysSchedule(almorzar)"))) 
print(KB2.query(pl.Expr("inTodaysSchedule(regar plantas)")))

# Crear consulta en pytholog
query = "isTask(X), isDailyTask(X)"

# Ejecutar consulta sobre la base de conocimientos KB2
#results = KB2.query(pl.Expr(query))
results = KB2.query(pl.Expr(query))

print("--- --- ---")
print(results)

# Verificar si hay resultados antes de imprimirlos
if results:
    # Recorrer los resultados de la consulta e imprimir los nombres de las tareas que cumplen la regla isDailyTask
    print("Tareas diarias:")
    for result in results:
        task_name = result.get("X")
        print(task_name)
else:
    print("No se encontraron tareas diarias.")

Preguntamos sobre una tarea que no esta en el dataframe
['Yes']
Preguntamos datos sobre una tarea que sí esta en el dataframe
['Yes']
Preguntamos por clasificaciones
['No']
['Yes']
Preguntamos datos sobre relaciones entre tareas
['No']
['No']
Distinguimos que tareas son las que se agregaron al horario de hoy
['No']
['Yes']
--- --- ---
['No']
Tareas diarias:


AttributeError: 'str' object has no attribute 'get'

Creamos un sistema de lógica difusa con la variable prioridad y se le asigna a cada una de las tareas.

In [ ]:
# Definir variables de entrada y salida
urgencia = ctrl.Antecedent(np.arange(1, 11, 1), 'urgencia')
prioridad = ctrl.Consequent(np.arange(0, 11, 1), 'prioridad')

# Definir los conjuntos difusos y las funciones de pertenencia correspondientes
urgencia['muy_baja'] = fuzz.trimf(urgencia.universe, [1, 1, 3])
urgencia['baja'] = fuzz.trimf(urgencia.universe, [1, 3, 5])
urgencia['media'] = fuzz.trimf(urgencia.universe, [3, 5, 7])
urgencia['alta'] = fuzz.trimf(urgencia.universe, [5, 7, 9])
urgencia['muy_alta'] = fuzz.trimf(urgencia.universe, [7, 10, 10])

prioridad['baja'] = fuzz.trimf(prioridad.universe, [0, 0, 5])
prioridad['media'] = fuzz.trimf(prioridad.universe, [0, 5, 10])
prioridad['alta'] = fuzz.trimf(prioridad.universe, [5, 10, 10])

# Definir las reglas difusas
regla1 = ctrl.Rule(urgencia['muy_baja'], prioridad['baja'])
regla2 = ctrl.Rule(urgencia['baja'], prioridad['baja'])
regla3 = ctrl.Rule(urgencia['media'], prioridad['media'])
regla4 = ctrl.Rule(urgencia['alta'], prioridad['alta'])
regla5 = ctrl.Rule(urgencia['muy_alta'], prioridad['alta'])

# Definir el sistema de control difuso
sistema = ctrl.ControlSystem([regla1, regla2, regla3, regla4, regla5])
asignar_prioridad = ctrl.ControlSystemSimulation(sistema)

def escribirPrioridad(prioridad):
    """_summary_

    Args:
        prioridad (_type_): _description_
    """
    # Imprimir la prioridad asignada en forma de cadena de texto
    if prioridad_tarea <= 2.5:
        print("La prioridad de la tarea es baja")
    elif prioridad_tarea <= 7.5:
        print("La prioridad de la tarea es media")
    else:
        print("La prioridad de la tarea es alta")




for clave, valor in dictTasks.items():
    print("asigno Prioridad para: " +clave +", que tiene urgencia: "+ str(round(valor.urgency,2)))

    # Asignar un valor de urgencia para la tarea de desayunar
    asignar_prioridad.input['urgencia'] = valor.urgency

    # Calcular el resultado del sistema de control difuso
    asignar_prioridad.compute()

    # Obtener la salida (prioridad)
    prioridad_tarea = asignar_prioridad.output['prioridad']
    print('La prioridad de la tarea es:', round(prioridad_tarea,2))
    valor.prioridad= prioridad_tarea
    escribirPrioridad(valor.prioridad)





asigno Prioridad para: bañarse, que tiene urgencia: 2
La prioridad de la tarea es: 1.94
La prioridad de la tarea es baja
asigno Prioridad para: hacer la tarea, que tiene urgencia: 7
La prioridad de la tarea es: 8.33
La prioridad de la tarea es alta
asigno Prioridad para: almorzar, que tiene urgencia: 4
La prioridad de la tarea es: 4.4
La prioridad de la tarea es media
asigno Prioridad para: hacer ejercicio, que tiene urgencia: 3
La prioridad de la tarea es: 1.67
La prioridad de la tarea es baja
asigno Prioridad para: estudiar para el parcial, que tiene urgencia: 9
La prioridad de la tarea es: 8.19
La prioridad de la tarea es alta
asigno Prioridad para: escuchar musica, que tiene urgencia: 1
La prioridad de la tarea es: 1.67
La prioridad de la tarea es baja
asigno Prioridad para: leer documentacion, que tiene urgencia: 5
La prioridad de la tarea es: 5.0
La prioridad de la tarea es media
asigno Prioridad para: sacar al perro, que tiene urgencia: 5
La prioridad de la tarea es: 5.0
La prio

Se describe cuales de las tareas que no están en el horario actual tienen mayor prioridad para que el usuario pueda priorizarlas lo mas pronto posible en caso de que tenga tiempo extra o deba cancelar otra actividad pendiente.

In [ ]:
# Crear un diccionario vacío para almacenar las tareas que no están en horario
nuevas_tareas = {}

# Iterar sobre las claves en dictTasks
for tarea in dictTasks.keys():
    
    # Comprobar si la tarea no está en horario
    if tarea not in horario:
        
        # Si no está en horario, agregarla al nuevo diccionario
        nuevas_tareas[tarea] = dictTasks[tarea]



print("En caso de no poder realizar alguna de las tareas en tu horario te recomendamos tomar alguna de alta prioridad,. \n a continuacion te las listamos en el orden recomendado: \n")

# Ordenar las tareas por prioridad (de mayor a menor)
tareas_ordenadas = sorted(nuevas_tareas.values(), key=lambda x: x.prioridad, reverse=True)

# Recorrer las tareas ordenadas e imprimir su prioridad
for tarea in tareas_ordenadas:
    print("- " +tarea.name + ", con una prioridad de: " + str(round(tarea.prioridad, 2) ))

En caso de no poder realizar alguna de las tareas en tu horario te recomendamos tomar alguna de alta prioridad,. 
 a continuacion te las listamos en el orden recomendado: 

- hacer la tarea, con una prioridad de: 8.33
- estudiar para el parcial, con una prioridad de: 8.19
- leer documentacion, con una prioridad de: 5.0
- almorzar, con una prioridad de: 4.4
- reunirse con amigos, con una prioridad de: 4.4
- ordenar el cuarto, con una prioridad de: 4.4
- bañarse, con una prioridad de: 1.94
- hacer estiramientos, con una prioridad de: 1.94
- hacer ejercicio, con una prioridad de: 1.67
- escuchar musica, con una prioridad de: 1.67
- ver netflix, con una prioridad de: 1.67


In [ ]:
for Task in dictTasks.values():
    print(str(Task.prioridad))

1.9444444444444444
8.333333333333334
4.404761904761905
1.6666666666666665
8.194444444444445
1.6666666666666665
5.000000000000001
5.000000000000001
1.9444444444444444
1.6666666666666665
4.404761904761905
8.333333333333334
4.404761904761905
5.595238095238095
